![https://pieriantraining.com/](../PTCenteredPurple.png)

In this lecture we will use boto3 to create and manage, as well as use, [Amazon EC2](https://aws.amazon.com/ec2/) instances

## Interacting with EC2 instances

## Create an EC2 instance
The first task we want to achieve is to create an ec2 instance using boto3. <br />
You can find the documentation on how to use the service ressources for ec2 [here](https://boto3.amazonaws.com/v1/documentation/api/latest/reference/services/ec2/service-resource/index.html)<br />
The function of interest for us is create_instances(**kwargs) [Doc](https://boto3.amazonaws.com/v1/documentation/api/latest/reference/services/ec2/service-resource/create_instances.html)

Before we start, we need to obtain an Amazon Machine Image: [AMI](https://docs.aws.amazon.com/AWSEC2/latest/UserGuide/AMIs.html) <br />
There are many ways to obtain such an image. However, for our cases, we can use a predefined image by ubuntu [Link](https://cloud-images.ubuntu.com/locator/ec2/)

Let's use the *us-east-1 ubuntu20.04* image using the *amd64* architecture. The corresponding ID is **ami-0a3db6a6bb59b68d3**

Next, let's define the [instance type](https://docs.aws.amazon.com/AWSEC2/latest/UserGuide/instance-types.html). You can find all instance types [here](https://aws.amazon.com/de/ec2/instance-types/).<br />
We can simply use one of the smallest instance (t2.nano) that costs $0.0058 / hour. [Here](https://aws.amazon.com/ec2/pricing/on-demand/) you can find all prices

Please note that some instances might become very expensive, so be very careful on what you select!

Hint: If you obtain an *UnauthorizedOperation* Error, you might need to update your IAM configuration and add the EC2 Access

In [6]:
import boto3

ec2 = boto3.resource("ec2", region_name="us-east-1")
instance = ec2.create_instances(
    ImageId="ami-0a3db6a6bb59b68d3", # Required
    MinCount=1, # Required
    MaxCount=1, # Required,
    InstanceType="t2.nano"
)

In [7]:
instance

[ec2.Instance(id='i-0461fa9315b2882a9')]

Congratulations! You just spawned an ec2 instance!
Feel free to head over to your aws console and navigate to EC2 -> Instances.

To inspect the attaced storage, head over to EC2-> Elastic Block Store

Of course we can also use boto3 to check what instances are running:

In [9]:
list(ec2.instances.all())

[ec2.Instance(id='i-0461fa9315b2882a9')]

In [15]:
instance[0].id

'i-0461fa9315b2882a9'

Using *client.describe_instances(InstanceIds=[string])* you can obtain all information about a particular instance

In [18]:
client = boto3.client("ec2", region_name="us-east-1")
instance_descr = client.describe_instances(InstanceIds=[instance[0].id])

In [19]:
instance_descr

{'Reservations': [{'Groups': [],
   'Instances': [{'AmiLaunchIndex': 0,
     'ImageId': 'ami-0a3db6a6bb59b68d3',
     'InstanceId': 'i-0461fa9315b2882a9',
     'InstanceType': 't2.nano',
     'LaunchTime': datetime.datetime(2023, 8, 8, 8, 40, 53, tzinfo=tzutc()),
     'Monitoring': {'State': 'disabled'},
     'Placement': {'AvailabilityZone': 'us-east-1e',
      'GroupName': '',
      'Tenancy': 'default'},
     'PrivateDnsName': 'ip-172-31-52-109.ec2.internal',
     'PrivateIpAddress': '172.31.52.109',
     'ProductCodes': [],
     'PublicDnsName': 'ec2-100-26-222-69.compute-1.amazonaws.com',
     'PublicIpAddress': '100.26.222.69',
     'State': {'Code': 16, 'Name': 'running'},
     'StateTransitionReason': '',
     'SubnetId': 'subnet-0e6aa81e2c43e33ad',
     'VpcId': 'vpc-0e332aaa00eaf3ed7',
     'Architecture': 'x86_64',
     'BlockDeviceMappings': [{'DeviceName': '/dev/sda1',
       'Ebs': {'AttachTime': datetime.datetime(2023, 8, 8, 8, 40, 54, tzinfo=tzutc()),
        'DeleteOnT

## Starting & Stopping instances
You can also use boto3 to start and stop instances.
The difference between isntance creation and instance starting is that you still have the attached memory available (and paying for it) and can restart it any time without creating a new isntance
Let's stop the above created instance first

In [33]:
instance_descr["Reservations"][0]["Instances"][0]["State"]

{'Code': 16, 'Name': 'running'}

In [34]:
client.stop_instances(InstanceIds=[instance[0].id])

{'StoppingInstances': [{'CurrentState': {'Code': 64, 'Name': 'stopping'},
   'InstanceId': 'i-0461fa9315b2882a9',
   'PreviousState': {'Code': 16, 'Name': 'running'}}],
 'ResponseMetadata': {'RequestId': '16998c73-ee08-4f12-b6c9-4f80db27a86a',
  'HTTPStatusCode': 200,
  'HTTPHeaders': {'x-amzn-requestid': '16998c73-ee08-4f12-b6c9-4f80db27a86a',
   'cache-control': 'no-cache, no-store',
   'strict-transport-security': 'max-age=31536000; includeSubDomains',
   'content-type': 'text/xml;charset=UTF-8',
   'content-length': '579',
   'date': 'Tue, 08 Aug 2023 08:52:50 GMT',
   'server': 'AmazonEC2'},
  'RetryAttempts': 0}}

In [43]:
client.describe_instances(InstanceIds=[instance[0].id])["Reservations"][0]["Instances"][0]["State"]

{'Code': 80, 'Name': 'stopped'}

Now let us restart it:

In [44]:
client.start_instances(InstanceIds=[instance[0].id])

{'StartingInstances': [{'CurrentState': {'Code': 0, 'Name': 'pending'},
   'InstanceId': 'i-0461fa9315b2882a9',
   'PreviousState': {'Code': 80, 'Name': 'stopped'}}],
 'ResponseMetadata': {'RequestId': '5c57beab-9f30-4170-801d-e67cfae544aa',
  'HTTPStatusCode': 200,
  'HTTPHeaders': {'x-amzn-requestid': '5c57beab-9f30-4170-801d-e67cfae544aa',
   'cache-control': 'no-cache, no-store',
   'strict-transport-security': 'max-age=31536000; includeSubDomains',
   'content-type': 'text/xml;charset=UTF-8',
   'content-length': '579',
   'date': 'Tue, 08 Aug 2023 08:55:24 GMT',
   'server': 'AmazonEC2'},
  'RetryAttempts': 0}}

In [46]:
client.describe_instances(InstanceIds=[instance[0].id])["Reservations"][0]["Instances"][0]["State"]

{'Code': 16, 'Name': 'running'}

## Terminating an instance
To remove your instance, you can use *client.terminate_instances(InstanceIds=[string])*

In [69]:
client.terminate_instances(InstanceIds=[instance[0].id])

{'TerminatingInstances': [{'CurrentState': {'Code': 32,
    'Name': 'shutting-down'},
   'InstanceId': 'i-0461fa9315b2882a9',
   'PreviousState': {'Code': 16, 'Name': 'running'}}],
 'ResponseMetadata': {'RequestId': '068bcaa8-2a17-4c82-93cf-31070738941b',
  'HTTPStatusCode': 200,
  'HTTPHeaders': {'x-amzn-requestid': '068bcaa8-2a17-4c82-93cf-31070738941b',
   'cache-control': 'no-cache, no-store',
   'strict-transport-security': 'max-age=31536000; includeSubDomains',
   'vary': 'accept-encoding',
   'content-type': 'text/xml;charset=UTF-8',
   'transfer-encoding': 'chunked',
   'date': 'Tue, 08 Aug 2023 09:13:22 GMT',
   'server': 'AmazonEC2'},
  'RetryAttempts': 0}}

Head over to aws and verify that your instace state is **terminated**.<br />
Also verify, that no volumes are shown in Elastic Block Store

## Access your instance via ssh using paramiko

Now that we know how to create an instance, we want to be able to access it via ssh. To do so we can use a python package called paramiko.
Before inspecting paramiko, let's create a new instance.
Before doing so, we should create an ssh key-pair which we assing to this newly created instance.
The public key is stored within AWS, and the private key is retured to us so we can use it. 

In [131]:
key_pair = client.create_key_pair(KeyName='first_aws_key_pair')
print(key_pair)


ClientError: An error occurred (InvalidKeyPair.Duplicate) when calling the CreateKeyPair operation: The keypair already exists

In [105]:
private_key = key_pair["KeyMaterial"]

In [106]:
with open('my-key-pair.pem', 'w') as f:
    f.write(private_key)


We now pass the KeyName to *create_instances*. <br >
In order to be able to access this instance, we need to attach a security group to it (otherwise all ports are blocked)

We can list all security groups using **describe_security_groups()**

In [116]:
security_groups = client.describe_security_groups()

You can see, that there is a group called launch-wizard-1, which contains the necessary permissions for ssh
Pass the GroupId of this group to **create_instance**()

In [119]:
security_groups["SecurityGroups"]

[{'Description': 'default VPC security group',
  'GroupName': 'default',
  'IpPermissions': [{'IpProtocol': '-1',
    'IpRanges': [],
    'Ipv6Ranges': [],
    'PrefixListIds': [],
    'UserIdGroupPairs': [{'GroupId': 'sg-0603fe742319bbc53',
      'UserId': '472948420345'}]}],
  'OwnerId': '472948420345',
  'GroupId': 'sg-0603fe742319bbc53',
  'IpPermissionsEgress': [{'IpProtocol': '-1',
    'IpRanges': [{'CidrIp': '0.0.0.0/0'}],
    'Ipv6Ranges': [],
    'PrefixListIds': [],
    'UserIdGroupPairs': []}],
  'VpcId': 'vpc-0e332aaa00eaf3ed7'},
 {'Description': 'launch-wizard-1 created 2022-07-22T20:10:04.156Z',
  'GroupName': 'launch-wizard-1',
  'IpPermissions': [{'FromPort': 80,
    'IpProtocol': 'tcp',
    'IpRanges': [{'CidrIp': '0.0.0.0/0'}],
    'Ipv6Ranges': [{'CidrIpv6': '::/0'}],
    'PrefixListIds': [],
    'ToPort': 80,
    'UserIdGroupPairs': []},
   {'FromPort': 8080,
    'IpProtocol': 'tcp',
    'IpRanges': [{'CidrIp': '0.0.0.0/0'}],
    'Ipv6Ranges': [],
    'PrefixListIds

In [121]:
ec2 = boto3.resource("ec2", region_name="us-east-1")
instance = ec2.create_instances(
    ImageId="ami-0a3db6a6bb59b68d3", # Required
    MinCount=1, # Required
    MaxCount=1, # Required,
    InstanceType="t2.nano",
    KeyName="first_aws_key_pair",
    SecurityGroupIds=["sg-0f32733d785714b4f"]  # Adjust that to your Group ID!
    
)

Now we are ready to tacke the ssh connection

In [47]:
import paramiko

1) Create paramiko ssh client
2) Use the **AutoAddPolicy** when the host key is missing (what it is in our case)
3) Pass the private key to paramiko
4) Grab the IP address of our instance

In [122]:
ssh = paramiko.SSHClient()
ssh.set_missing_host_key_policy(paramiko.AutoAddPolicy())
private_key = paramiko.RSAKey(filename="my-key-pair.pem")


In [123]:
ip = client.describe_instances(InstanceIds=[instance[0].id])["Reservations"][0]["Instances"][0]["PublicIpAddress"]

In [124]:
ip

'54.160.81.220'

In [129]:
ssh.connect(hostname=ip, username="ubuntu", pkey=private_key)

stdin , stdout, stderr = ssh.exec_command("ls")
print(stdout.read())

stdin , stdout, stderr = ssh.exec_command("mkdir TestDir")
print(stdout.read())

stdin , stdout, stderr = ssh.exec_command("ls")
print(stdout.read())

ssh.close()


b''
b''
b'TestDir\n'


**Don't forget to terminate your instance!!**

In [130]:
client.terminate_instances(InstanceIds=[instance[0].id])

{'TerminatingInstances': [{'CurrentState': {'Code': 32,
    'Name': 'shutting-down'},
   'InstanceId': 'i-0b9f120993b01c975',
   'PreviousState': {'Code': 16, 'Name': 'running'}}],
 'ResponseMetadata': {'RequestId': '7e42dbf9-c7b9-4912-b286-44831dd745a6',
  'HTTPStatusCode': 200,
  'HTTPHeaders': {'x-amzn-requestid': '7e42dbf9-c7b9-4912-b286-44831dd745a6',
   'cache-control': 'no-cache, no-store',
   'strict-transport-security': 'max-age=31536000; includeSubDomains',
   'vary': 'accept-encoding',
   'content-type': 'text/xml;charset=UTF-8',
   'transfer-encoding': 'chunked',
   'date': 'Tue, 08 Aug 2023 09:59:06 GMT',
   'server': 'AmazonEC2'},
  'RetryAttempts': 0}}